In [ ]:
import zipfile
import os
print(os.getcwd())
os.environ["KAGGLE_CONFIG_DIR"] = "/content"

/content


In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"prathameshpotabatti","key":"78acef74d5d6994837e54738cb8d10b7"}'}

In [ ]:
! chmod 600 ./kaggle.json

In [ ]:
! kaggle datasets download -d nickyazdani/license-plate-text-recognition-dataset

 95% 86.0M/90.8M [00:00<00:00, 204MB/s]
100% 90.8M/90.8M [00:00<00:00, 207MB/s]


In [ ]:
zip_ref = zipfile.ZipFile('license-plate-text-recognition-dataset.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
! pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders

splitfolders.ratio("cropped_lps/", output="output", seed=1337, ratio=(.8, .2), group_prefix=None)

Copying files: 20000 files [00:03, 5662.04 files/s]


In [ ]:
train_data = f"{os.getcwd()}/output/train/"
val_data = f"{os.getcwd()}/output/val/"

In [ ]:
letters = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [ ]:
import pandas as pd

df = pd.read_csv("lpr.csv")
df.iloc[1]["labels"]

'V95246'

In [ ]:
# def reshapeAndNormalize(w, h, img):
#     if w < 32:
#         add_zeros = np.ones((32-w, h))*255
#         img = np.concatenate((img, add_zeros))

#     if h < 128:
#         add_zeros = np.ones((32, 128-h))*255
#         img = np.concatenate((img, add_zeros), axis=1)
#     img = np.expand_dims(img , axis = 2)
      
#     # Normalize each image
#     img = img/255.

#     return img

In [ ]:
import fnmatch
import cv2
import numpy as np

char_list = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"

def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)
        
    return dig_lst

# lists for training dataset
training_img = []
training_txt = []
train_input_length = []
train_label_length = []
orig_txt = []
 
#lists for validation dataset
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []

path = f"{os.getcwd()}/cropped_lps/"
max_label_len = 10
i = 1 
flag = 0

for root, dirnames, filenames in os.walk(path):
 
    for f_name in fnmatch.filter(filenames, '*.jpg'):
        # read input image and convert into gray scale image
        img = cv2.cvtColor(cv2.imread(os.path.join(root, f_name)), cv2.COLOR_BGR2GRAY)   
 
        # convert each image of shape (32, 128, 1)
        w, h = img.shape
        if h > 128 or w > 32:
            continue
        if w < 32:
            add_zeros = np.ones((32-w, h))*255
            img = np.concatenate((img, add_zeros))
 
        if h < 128:
            add_zeros = np.ones((32, 128-h))*255
            img = np.concatenate((img, add_zeros), axis=1)
        img = np.expand_dims(img , axis = 2)
        
        # Normalize each image
        img = img/255.

        # split the 20000 data into validation and training dataset as 20% and 80% respectively
        if i%5 == 0:
            valid_orig_txt.append(df.iloc[i]["labels"])
            valid_label_length.append(len(df.iloc[i]["labels"]))
            valid_input_length.append(10)
            valid_img.append(img)
            valid_txt.append(encode_to_labels(df.iloc[i]["labels"]))
        else:
            orig_txt.append(df.iloc[i]["labels"])   
            train_label_length.append(len(df.iloc[i]["labels"]))
            train_input_length.append(10)
            training_img.append(img)
            training_txt.append(encode_to_labels(df.iloc[i]["labels"]))
        
        # break the loop if total data is 20000
        if i == 20000:
            flag = 1
            break
        i+=1
    if flag == 1:
        break

In [ ]:
img_w = 32
img_h = 128
img_c = 1
num_classes = len(char_list) + 1

In [ ]:
import keras
from keras.layers import Dense, Input, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Reshape, Lambda, MaxPool2D
from keras.layers import BatchNormalization, Dropout, LSTM, Bidirectional
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def ctc_loss_function(args):
    y_pred, labels, input_length, label_length = args
    return keras.backend.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
def Image_text_recogniser_model(stage="train",drop_out_rate=0.35):
    """
    Builds the model by taking in the stage variable which specifes the stage,
    if the stage is training: model takes inputs required for computing ctc_batch_cost function
    else : model takes input as images which is used for prediction
    """
    
    if keras.backend.image_data_format() == 'channels_first':
        input_shape = (1, img_w, img_h)
    else:
        input_shape = (img_w, img_h, 1)
        
    model_input=Input(shape=input_shape,name='img_input',dtype='float32')

    conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(model_input)
    pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
    
    conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
    pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
    
    conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
    
    conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
    pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
    
    conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
    batch_norm_5 = BatchNormalization()(conv_5)
    
    conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
    batch_norm_6 = BatchNormalization()(conv_6)
    pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
    
    conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)

    squeezed = Lambda(lambda x: keras.backend.squeeze(x, 1))(conv_7)
    # RNN layer
    model=Bidirectional(LSTM(128, return_sequences=True, kernel_initializer='he_normal'), merge_mode='sum')(squeezed)
    model=Bidirectional(LSTM(128, return_sequences=True, kernel_initializer='he_normal'), merge_mode='concat')(model)

    # transforms RNN output to character activations:
    model = Dense(num_classes, kernel_initializer='he_normal',name='dense2')(model)
    y_pred = Activation('softmax', name='softmax')(model)

    labels = Input(name='ground_truth_labels', shape=[max_label_len], dtype='float32') 
    input_length = Input(name='input_length', shape=[1], dtype='int64') 
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    #CTC loss function
    loss_out = Lambda(ctc_loss_function, output_shape=(1,),name='ctc')([y_pred, labels, input_length, label_length])

    if stage=='train':
        return model_input,y_pred,keras.Model(inputs=[model_input, labels, input_length, label_length], outputs=loss_out)
    else:
        return keras.Model(inputs=[model_input], outputs=y_pred)

In [ ]:
model_input, y_pred, crnn = Image_text_recogniser_model()

In [ ]:
crnn.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_input (InputLayer)         [(None, 32, 128, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_63 (Conv2D)             (None, 32, 128, 64)  640         ['img_input[0][0]']              
                                                                                                  
 max_pooling2d_36 (MaxPooling2D  (None, 16, 64, 64)  0           ['conv2d_63[0][0]']              
 )                                                                                                
                                                                                            

In [ ]:
from keras.optimizers import Adam

crnn.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')

filepath="best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

In [ ]:
training_img = np.array(training_img)
train_input_length = np.array(train_input_length)
train_label_length = np.array(train_label_length)

valid_img = np.array(valid_img)
valid_input_length = np.array(valid_input_length)
valid_label_length = np.array(valid_label_length)

In [ ]:
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

In [ ]:
batch_size = 10
epochs = 10
crnn.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length], y=np.zeros(len(training_img)), batch_size=batch_size, epochs = epochs, validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], [np.zeros(len(valid_img))]), verbose = 1, callbacks = callbacks_list)

Epoch 1/10
11/11 [==============================] - ETA: 0s - loss: 22.8891
Epoch 1: val_loss improved from inf to 20.65809, saving model to best_model.hdf5
11/11 [==============================] - 19s 1s/step - loss: 22.8891 - val_loss: 20.6581
Epoch 2/10
11/11 [==============================] - ETA: 0s - loss: 20.4791
Epoch 2: val_loss improved from 20.65809 to 20.31706, saving model to best_model.hdf5
11/11 [==============================] - 10s 866ms/step - loss: 20.4791 - val_loss: 20.3171
Epoch 3/10
11/11 [==============================] - ETA: 0s - loss: 20.1701
Epoch 3: val_loss did not improve from 20.31706
11/11 [==============================] - 10s 931ms/step - loss: 20.1701 - val_loss: 20.6282
Epoch 4/10
11/11 [==============================] - ETA: 0s - loss: 19.9983
Epoch 4: val_loss did not improve from 20.31706
11/11 [==============================] - 10s 911ms/step - loss: 19.9983 - val_loss: 20.6207
Epoch 5/10
11/11 [==============================] - ETA: 0s - loss: 

In [ ]:
crnn_test = Image_text_recogniser_model("test")

In [ ]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

In [ ]:
def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

In [ ]:
def detect(img_rgb):
    img = img_rgb.copy()
    input_height = img_rgb.shape[0]
    input_width = img_rgb.shape[1]
    hsv_frame = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2HSV)

    # yellow color
    low_yellow = np.array([20, 100, 100])
    high_yellow = np.array([30, 255, 255])
    yellow_mask = cv2.inRange(hsv_frame, low_yellow, high_yellow)
    yellow = cv2.bitwise_and(yellow_mask, yellow_mask, mask=yellow_mask)

    cv2.imwrite("temp/steps/1_yellow_color_detection.png", yellow)
    # Close morph
    k = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(yellow, cv2.MORPH_CLOSE, k)

    cv2.imwrite("temp/steps/2_closing_morphology.png", closing)
    # Detect yellow area
    contours, hierarchy = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # List of final crops
    crops = []

    # Loop over contours and find license plates
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        # Conditions on crops dimensions and area
        if h*6 > w > 2 * h and h > 0.1 * w and w * h > input_height * input_width * 0.0001:

            # Make a crop from the RGB image, the crop is slided a bit at left to detect bleu area
            crop_img = img_rgb[y:y + h, x-round(w/10):x]
            crop_img = crop_img.astype('uint8')

            # Compute bleu color density at the left of the crop
            # Bleu color condition
            try:
                hsv_frame = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
                low_bleu = np.array([100,150,0])
                high_bleu = np.array([140,255,255])
                bleu_mask = cv2.inRange(hsv_frame, low_bleu, high_bleu)
                bleu_summation = bleu_mask.sum()

            except:
                bleu_summation = 0

            # Condition on bleu color density at the left of the crop
            if bleu_summation > 550:

                # Compute yellow color density in the crop
                # Make a crop from the RGB image
                imgray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
                crop_img_yellow = img_rgb[y:y + h, x:x+w]
                crop_img_yellow = crop_img_yellow.astype('uint8')

                # Detect yellow color
                hsv_frame = cv2.cvtColor(crop_img_yellow, cv2.COLOR_BGR2HSV)
                low_yellow = np.array([20, 100, 100])
                high_yellow = np.array([30, 255, 255])
                yellow_mask = cv2.inRange(hsv_frame, low_yellow, high_yellow)

                # Compute yellow density
                yellow_summation = yellow_mask.sum()

                # Condition on yellow color density in the crop
                if yellow_summation > 255*crop_img.shape[0]*crop_img.shape[0]*0.4:

                    # Make a crop from the gray image
                    crop_gray = imgray[y:y + h, x:x + w]
                    crop_gray = crop_gray.astype('uint8')

                    # Detect chars inside yellow crop with specefic dimension and area
                    th = cv2.adaptiveThreshold(crop_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
                    contours2, hierarchy = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                    # Init number of chars
                    chars = 0
                    for c in contours2:
                        area2 = cv2.contourArea(c)
                        x2, y2, w2, h2 = cv2.boundingRect(c)
                        if w2 * h2 > h * w * 0.01 and h2 > w2 and area2 < h * w * 0.9:
                            chars += 1

                    # Condition on the number of chars
                    if 20 > chars > 4:
                        rect = cv2.minAreaRect(cnt)
                        box = cv2.boxPoints(rect)
                        box = np.int0(box)
                        pts = np.array(box)
                        warped = four_point_transform(img, pts)
                        crops.append(warped)

                        # Using cv2.putText() method
                        #img_rgb = cv2.putText(img_rgb, 'LP', (x, y), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 2, cv2.LINE_AA)
                        #print(pts)
                        cv2.drawContours(img_rgb, [box], 0, (0, 0, 255), 1)

    return img_rgb, crops, pts

In [ ]:
# import numpy as np
# from google.colab import files
# import keras.utils as image
# from PIL import Image

# from google.colab.patches import cv2_imshow

# uploaded = files.upload()
# print(uploaded)



# img_rgb, crops, pts = detect(uploaded)

# cv2_imshow(img_rgb)
# for fn in uploaded.keys():
#     input_image = cv2.imread(uploaded)
#     detection, crops, box1 = detect(input_image)

In [ ]:
# crnn_test.load_weights('best_model.hdf5')
 
# predict outputs on validation images
prediction = crnn_test.predict(valid_img[:10])

# use CTC decoder
out = keras.backend.get_value(keras.backend.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])

# see the results
i = 0
for x in out:
    print("original_text =  ", valid_orig_txt[i])
    print("predicted text = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(char_list[int(p)], end = '')       
    print('\n')
    i+=1

1/1 [==============================] - 1s 1s/step
original_text =   0195SM
predicted text = 4XEN

original_text =   H66544
predicted text = 4XEN

original_text =   W35478
predicted text = 4XEN

original_text =   A04129
predicted text = 4XEN

original_text =   NC6179
predicted text = 4XEN

original_text =   TB5888
predicted text = 4XEN

original_text =   2749UY
predicted text = 4XEN

original_text =   QD5532
predicted text = 4XEN

original_text =   2746BC
predicted text = 4XEN

original_text =   3561JB
predicted text = 4XEN



In [ ]:
img = cv2.imread("sample1.png")


In [ ]:
# predict outputs on validation images
# input_imgs = []
input_img = cv2.cvtColor(cv2.imread("sample1.png"), cv2.COLOR_BGR2GRAY)
up_points = (128, 32)

resized_up = cv2.resize(input_img, up_points)
resized_up = np.expand_dims(resized_up, axis=-1)
resized_up = np.array(resized_up)[np.newaxis, ...]
# resized_up = np.expand_dims(resized_up, axis=0)
print(resized_up.shape)
prediction = crnn_test.predict(resized_up)

# use CTC decoder
out = keras.backend.get_value(keras.backend.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])

# see the results
for x in out:
    print("predicted text = ", end = '')
    for p in x:
        if int(p) != -1:
            print(char_list[int(p)], end = '')
    print('\n')

(1, 32, 128, 1)


1/1 [==============================] - 1s 1s/step
predicted text = YXYXYXZ

